In [1]:
import corpus as corpus_class
import categories, filters, vocabulary_builders
from feature_extractors import multinomial_model, tfidf
from filters import std_filters

import numpy as np

In [2]:
qfile_train = 'question_train.csv'
qcatfile_train = 'question_category_train.csv'
catfile = 'category.csv'
qfile_test = 'question_test.csv'

In [3]:
filtees = std_filters()

In [4]:
corpus = corpus_class.corpus( categories.categories() )
corpus.load(qfile_train, qcatfile_train)
corpus.process(**filtees, corpus_size=-1)

In [5]:
spls = corpus.spell_checker
(spls[10]).num_corrected
for i in range(len(corpus.cats)):
    print(len(spls[i].vocabulary), "\t", spls[i].num_corrected)

AttributeError: 'corpus' object has no attribute 'spell_checker'

In [6]:
corpus.simple_split(0.3)

In [7]:
corpus.make_features(-1)

In [8]:
print(corpus)

14 categories. 
- loaded from file: True
	 14417 docuemnts loaded from file. 
- processed: True
	 sentence_filters: ['punctuation_filter'] 
	 word_filters: ['small_word_filter', 'stopword_filter', 'stemming_filter'] 
- corpus in simple split:
	 Training-set, Test-set size: (10091, 4326) 
- made numeric features: True
	 vocabulary_builder, M: ig_based_non_uniform, -1 
	 feature_extractor: multinomial_model 



In [9]:
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

clf_nb = MultinomialNB(alpha=0.1)
clf_lr = LogisticRegression(C=3.0)

In [10]:
bcf = VotingClassifier(estimators=[('lr', clf_lr), ('nb', clf_nb)], voting='soft')

In [11]:
bcf.fit(corpus.X_tr, corpus.y_tr);

In [12]:
bcf.score(corpus.X_te, corpus.y_te)

0.6137309292649098

In [48]:
import spell_checker
import importlib
from collections import Counter

In [49]:
importlib.reload(spell_checker)

<module 'spell_checker' from '/Users/paulhager/Dropbox/studium16-17/ml_project/spell_checker.py'>

In [50]:
spell_checker.candidates("hans")

{('ans', 0.10000000000000009),
 ('gans', 0.7),
 ('hams', 0.7),
 ('han', 0.3307692307692309),
 ('hanp', 0.7),
 ('hanq', 0.7),
 ('hanr', 0.7),
 ('hans', 1),
 ('hanß', 0.7),
 ('haos', 0.7),
 ('has', 0.2666666666666667),
 ('haös', 0.7),
 ('hbns', 0.7),
 ('hcns', 0.7),
 ('hns', 0.19090909090909103),
 ('häns', 0.7),
 ('ians', 0.7)}